In [1]:
import os 
import glob
import numpy as np 
import pandas as pd

os.chdir('/mnt/BioAdHoc/Groups/vd-ay/hichip-db-loop-calling/')
samplesheet_dir = 'results/samplesheets/chipseq/'
batch = '2024-rerun-samples'

rerun_samples = ['BMDC.GSE185880.Mus_Musculus.H3K27ac.b1',
            'BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1',
            'BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27ac.b1',
            'BMDC-LPS.GSE185880.Mus_Musculus.H3K27ac.b1',
            'NK92.GSE227660.Homo_Sapiens.H3K27ac.b1',
            'NK92-SAHA.GSE227660.Homo_Sapiens.H3K27ac.b1']

rerun_names = ['BMDC', 'BMDC-JSH', 'BMDC-JSH-JSH-LPS', 'BMDC-LPS', 'NK92', 'NK92-SAHA']

## Load the GSM Data

In [2]:
fn = os.path.join(samplesheet_dir, 'ChIP-seq-Tracker-Batch2-8.16.22-10.23.23-Step0.HiChIP-to-ChIP-seq-Map.tsv')
data = pd.read_table(fn, skiprows=1)
data = data.loc[data['Has ChIP-seq?'] == 'Yes']

# extracting missing samples
data = data.loc[data['Sample Name'].isin(rerun_names)]

In [3]:
chipseq_gse_lists = data['GSM ID(s)'].dropna().tolist() + data['Control GSM(s)'].dropna().tolist()
chipseq_gsms = []
for gse_list in chipseq_gse_lists:
    gsm_ids = [x.strip() for x in gse_list.split(',')]
    chipseq_gsms.extend(gsm_ids)
    
chipseq_gsms = sorted(set(chipseq_gsms))

In [4]:
data = data.loc[data['Has ChIP-seq?'] == 'Yes']

In [5]:
data.shape

(12, 15)

Query the below within the SRA Run Selector: https://www.ncbi.nlm.nih.gov/Traces/study/

In [6]:
','.join(chipseq_gsms)

'GSM5625199,GSM5625200,GSM5625207,GSM5625208,GSM6620326,GSM6620327,GSM6620328,GSM6620329,GSM6620330,GSM6620331,GSM6620332,GSM6620333,GSM7104825,GSM7104826,GSM7104827,GSM7104830,GSM7104831'

In [7]:
len(chipseq_gsms)

17

## Parse the Queried Data

In [8]:
# load the data
sra_table_fn = os.path.join(samplesheet_dir, 'chipseq.{}.SraRunTable.txt'.format(batch))
gsm_to_srr_data = pd.read_table(sra_table_fn, sep=',')
gsm_to_srr_data = gsm_to_srr_data[['Sample Name', 'Run']]
gsm_to_srr_data.columns = ['gsm_id', 'srr_id']
gsm_to_srr_data.sort_values('gsm_id', inplace=True)

In [9]:
# manually adding missing samples
adding_missing_entries = [['GSM7336681', 'SRR16538542'],
                          ['GSM7336682', 'SRR16538550'],
                          ['GSM7336684', 'SRR16538541'],
                          ['GSM7336685', 'SRR16538549'],
]
adding_missing_entries = pd.DataFrame(adding_missing_entries, columns=gsm_to_srr_data.columns.tolist())
gsm_to_srr_data = pd.concat([gsm_to_srr_data, adding_missing_entries], axis=0)

## Check for Discrepancies

In [10]:
original_gses = set(chipseq_gsms)

In [11]:
queried_gses = set(gsm_to_srr_data.gsm_id.unique())

In [12]:
len(queried_gses)

21

In [13]:
original_gses.difference(queried_gses)

set()

Not sure how I got these extra stragglers 

In [14]:
queried_gses.difference(original_gses)

{'GSM7336681', 'GSM7336682', 'GSM7336684', 'GSM7336685'}

In [15]:
gsm_to_srr_data.gsm_id.value_counts()

gsm_id
GSM5625199    1
GSM5625200    1
GSM5625207    1
GSM5625208    1
GSM6620326    1
GSM6620327    1
GSM6620328    1
GSM6620329    1
GSM6620330    1
GSM6620331    1
GSM6620332    1
GSM6620333    1
GSM7104825    1
GSM7104826    1
GSM7104827    1
GSM7104830    1
GSM7104831    1
GSM7336681    1
GSM7336682    1
GSM7336684    1
GSM7336685    1
Name: count, dtype: int64

## Create the Samplesheet for Downloading

In [16]:
# get unique SRRs 
srr_ids = sorted(gsm_to_srr_data.srr_id.unique())

In [17]:
srr_ids

['SRR16323879',
 'SRR16323880',
 'SRR16323887',
 'SRR16323888',
 'SRR16538541',
 'SRR16538542',
 'SRR16538549',
 'SRR16538550',
 'SRR21839422',
 'SRR21839423',
 'SRR21839424',
 'SRR21839425',
 'SRR21839426',
 'SRR21839427',
 'SRR21839428',
 'SRR21839429',
 'SRR23908058',
 'SRR23908059',
 'SRR23908062',
 'SRR23908063',
 'SRR23908064']

In [18]:
srr_ids_fn = os.path.join(samplesheet_dir, 'chipseq.srr_ids.{}.txt'.format(batch))
with open(srr_ids_fn, 'w') as fw:
    fw.write('\n'.join(srr_ids))

## Create the Samplesheet for Concatination

In [19]:
# get Combinations of SRRs that should be concatinated
gsm_srr_combos_data = []

# main GSMs
for gsm_ids_combo in data['GSM ID(s)'].tolist():
    gsm_ids = gsm_ids_combo.replace(' ', '').split(',')
    gsm_combo_reps = [gsm_ids_combo] * len(gsm_ids)
    split_data = list(zip(gsm_combo_reps, gsm_ids))
    gsm_srr_combos_data.extend(split_data)    
    
# control GSMs
for gsm_ids_combo in data['Control GSM(s)'].tolist():
    
    if type(gsm_ids_combo) == str:
        gsm_ids = gsm_ids_combo.replace(' ', '').split(',')
        gsm_combo_reps = [gsm_ids_combo] * len(gsm_ids)
        split_data = list(zip(gsm_combo_reps, gsm_ids))
    else:
        split_data = [[np.nan, np.nan]]
    gsm_srr_combos_data.extend(split_data)    

In [20]:
gsm_srr_combos_data = pd.DataFrame(gsm_srr_combos_data, columns=['GSM ID(s)', 'split_gsm_ids']).drop_duplicates()
gsm_srr_combos_data = gsm_srr_combos_data.merge(gsm_to_srr_data, left_on='split_gsm_ids', right_on='gsm_id', how='outer', indicator=True)
gsm_srr_combos_data = gsm_srr_combos_data.loc[~gsm_srr_combos_data.gsm_id.isna()]

In [21]:
gsm_srr_combos_data_agg = gsm_srr_combos_data.groupby('GSM ID(s)').agg({'srr_id': '-'.join})
srr_combos = sorted(gsm_srr_combos_data_agg.srr_id.unique())

In [22]:
srr_combos_fn = os.path.join(samplesheet_dir, 'chipseq.srr_id.combos.{}.txt'.format(batch))
with open(srr_combos_fn, 'w') as fw:
    fw.write('\n'.join(srr_combos))

In [23]:
len(srr_combos)

9

In [24]:
srr_combos

['SRR16323879',
 'SRR16323880',
 'SRR16323887-SRR21839429-SRR21839428',
 'SRR16323888-SRR21839423-SRR21839422',
 'SRR21839425-SRR21839424',
 'SRR21839427-SRR21839426',
 'SRR23908059-SRR23908058',
 'SRR23908063-SRR23908062',
 'SRR23908064']

In [25]:
srr_combos_fn

'results/samplesheets/chipseq/chipseq.srr_id.combos.2024-rerun-samples.txt'

## Create the Samplesheet for Alignment of Control SRRs

#### Generate a samplesheet that contains the organism

In [26]:
# extracting gsms
df1 = data[['GSM ID(s)', 'Organism']].rename(columns={'GSM ID(s)': 'gsm_ids'})
df1['sample_type'] = 'main'
df2 = data[['Control GSM(s)', 'Organism']].rename(columns={'Control GSM(s)': 'gsm_ids'})
df2['sample_type'] = 'control'

# concating the gsm dfs
gsm_to_organism = pd.concat([df1, df2], axis=0).dropna().drop_duplicates()

# adding srr information
gsm_to_organism = gsm_to_organism.merge(gsm_srr_combos_data_agg, left_on='gsm_ids', right_index=True)

# extract control only
gsm_to_organism_control = gsm_to_organism.loc[gsm_to_organism.sample_type == 'control']

# adding reference genome information
human_subset = gsm_to_organism_control.loc[gsm_to_organism_control['Organism'] == 'Homo Sapiens']
mouse_subset = gsm_to_organism_control.loc[gsm_to_organism_control['Organism'] == 'Mus Musculus']

grch38_subset = human_subset.copy().sort_values(['srr_id'])
grch38_subset.loc[:, 'ref_genome'] = 'grch38'

# t2t_subset = human_subset.copy().sort_values(['srr_id'])
# t2t_subset.loc[:, 'ref_genome'] = 't2t'

mm10_subset = mouse_subset.copy().sort_values(['srr_id'])
mouse_subset.loc[:, 'ref_genome'] = 'mm10'

# concating all the subsets for a final samplesheet
# gsm_to_organism_control = pd.concat([grch38_subset, mouse_subset, t2t_subset])
gsm_to_organism_control = pd.concat([grch38_subset, mouse_subset])

/mnt/BioHome/jreyna/tmp/ipykernel_1489580/1558082514.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mouse_subset.loc[:, 'ref_genome'] = 'mm10'


In [27]:
# extract srr and ref_genome
gsm_to_organism_control = gsm_to_organism_control[['srr_id', 'ref_genome']]

In [28]:
samplesheet_path = os.path.join(samplesheet_dir, 'chipseq.srr_ids_to_ref_genome.combos.{}.txt'.format(batch))
gsm_to_organism_control.to_csv(samplesheet_path, sep='\t', header=False, index=False)

In [29]:
samplesheet_path

'results/samplesheets/chipseq/chipseq.srr_ids_to_ref_genome.combos.2024-rerun-samples.txt'

## Create the Samplesheet for Running ChIPLine

In [31]:
# process only uniqe GSM combos
chipline_ss = data[['Sample Name', 'GSE ID For ChIP-seq Data', 'Organism', 'Pulldown', 'GSM ID(s)', 'Control GSM(s)']]
chipline_ss = chipline_ss.drop_duplicates(subset=['GSM ID(s)'])

In [32]:
# add the srr information
chipline_ss = chipline_ss.merge(gsm_srr_combos_data_agg, left_on='GSM ID(s)', right_index=True, how='left')
chipline_ss.rename(columns={'srr_id': 'Main SRR(s)'}, inplace=True)
chipline_ss = chipline_ss.merge(gsm_srr_combos_data_agg, left_on='Control GSM(s)', right_index=True, how='left')
chipline_ss.rename(columns={'srr_id': 'Control SRR(s)'}, inplace=True)

In [33]:
# add organism infor
chipline_ss.loc[:, 'Organism'] = chipline_ss.loc[:, 'Organism'].str.replace('Homo Sapiens', 'Homo_Sapiens')
chipline_ss.loc[:, 'Organism'] = chipline_ss.loc[:, 'Organism'].str.replace('Mus Musculus', 'Mus_Musculus')

In [34]:
# adding the biorep information 
chipline_ss_tmp_list = []
sort_cols = ['GSE ID For ChIP-seq Data', 'Sample Name', 'Organism', 'Pulldown', 'GSM ID(s)', 'Control GSM(s)']
chipline_ss = chipline_ss.sort_values(sort_cols)
for grp, grp_df in chipline_ss.groupby(['GSE ID For ChIP-seq Data', 'Sample Name', 'Organism', 'Pulldown']):
    grp_df.loc[:, 'Rep No.'] = ['b{}'.format(x) for x in range(1, grp_df.shape[0] + 1)]
    chipline_ss_tmp_list.append(grp_df)
chipline_ss = pd.concat(chipline_ss_tmp_list)

In [35]:
# add the standard sample name 
def create_std_sample_name(sr):
    name = '{}.{}.{}.{}.{}'.format(sr['Sample Name'], sr['GSE ID For ChIP-seq Data'],
                                   sr['Organism'], sr['Pulldown'], sr['Rep No.'])
    return(name)
chipline_ss.loc[:, 'chipseq_std_sample_name'] = chipline_ss.apply(create_std_sample_name, axis=1)

In [36]:
chipline_ss

,Sample Name,GSE ID For ChIP-seq Data,Organism,Pulldown,GSM ID(s),Control GSM(s),Main SRR(s),Control SRR(s),Rep No.,chipseq_std_sample_name
62,BMDC,GSE185880,Mus_Musculus,H3K27ac,"GSM5625207, GSM6620326, GSM6620327",GSM5625199,SRR16323887-SRR21839429-SRR21839428,SRR16323879,b1,BMDC.GSE185880.Mus_Musculus.H3K27ac.b1
64,BMDC-JSH,GSE185880,Mus_Musculus,H3K27ac,"GSM6620328, GSM6620329",NaN,SRR21839427-SRR21839426,NaN,b1,BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1
66,BMDC-JSH-JSH-LPS,GSE185880,Mus_Musculus,H3K27ac,"GSM6620330, GSM6620331",NaN,SRR21839425-SRR21839424,NaN,b1,BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27a...
68,BMDC-LPS,GSE185880,Mus_Musculus,H3K27ac,"GSM5625208, GSM6620332, GSM6620333",GSM5625200,SRR16323888-SRR21839423-SRR21839422,SRR16323880,b1,BMDC-LPS.GSE185880.Mus_Musculus.H3K27ac.b1
85,NK92,GSE227660,Homo_Sapiens,H3K27ac,"GSM7104826, GSM7104827",GSM7104825,SRR23908063-SRR23908062,SRR23908064,b1,NK92.GSE227660.Homo_Sapiens.H3K27ac.b1
87,NK92-SAHA,GSE227660,Homo_Sapiens,H3K27ac,"GSM7104830, GSM7104831",GSM7104825,SRR23908059-SRR23908058,SRR23908064,b1,NK92-SAHA.GSE227660.Homo_Sapiens.H3K27ac.b1


In [37]:
# organize the reference genome of samples
chipline_ss.loc[:, 'Ref Genome'] = chipline_ss['Organism'].map({'Homo_Sapiens': 'grch38',
                                                                 'Mus_Musculus': 'mm10'})
# add T2T samples
chipline_ss_grps = chipline_ss.groupby('Ref Genome')
# t2t_chipline_ss = chipline_ss_grps.get_group('grch38').copy(deep=True)
# t2t_chipline_ss.loc[:, 'Ref Genome'] = 't2t'

In [38]:
# generate teh final chipline samplesheet
#chipline_ss = pd.concat([chipline_ss, t2t_chipline_ss])
chipline_ss = pd.concat([chipline_ss])

In [39]:
# organize the columns 
chipline_ss = chipline_ss[['chipseq_std_sample_name', 'Main SRR(s)', 'Control SRR(s)', 'Ref Genome',
                           'Sample Name', 'GSE ID For ChIP-seq Data',
                            'Organism', 'Pulldown', 'Rep No.', 'GSM ID(s)', 'Control GSM(s)']]

In [40]:
# finalize and save
chipline_ss.fillna('N/A', inplace=True)
chipline_ss_fn = os.path.join(samplesheet_dir, 'chipseq.chipline.{}.txt'.format(batch))
chipline_ss.to_csv(chipline_ss_fn, sep='\t', index=False, header=False)

In [41]:
chipline_ss.shape

(6, 11)

In [43]:
chipline_ss_fn

'results/samplesheets/chipseq/chipseq.chipline.2024-rerun-samples.txt'

## Add the ChIP-seq file that finally maps between HiChIP and ChIP-seq

In [ ]:
mapper = data.copy()
mapper.rename(columns={'Replicate Serial No': 'Rep No.'}, inplace=True)
mapper.loc[:, 'Organism'] = mapper.loc[:, 'Organism'].str.replace(' ' , '_') 
mapper.loc[:, 'Rep No.'] = ['b{}'.format(x) for x in mapper.loc[:, 'Rep No.'].astype(int)]
mapper.loc[:, 'chipseq_std_sample_name'] = mapper.apply(create_std_sample_name, axis=1)

def get_chipseq_path(std_sample_name):
    
    res_dir = 'results/peaks/chipline_v2/'
    sample_tmpl = os.path.join(res_dir, '{0}/MACS2_Ext_*/{0}.macs2_peaks.narrowPeak_Q0.01filt'.format(std_sample_name))
    print(sample_tmpl)
    peak_fns = glob.glob(sample_tmpl)
    
    if len(peak_fns) == 1:
        return(peak_fns[0])
    elif len(peak_fns) > 1:
        return('Found multiple files')
    else:
        return('Could not find a matching file')

mapper.loc[:, 'chipseq_path'] = mapper.loc[:, 'chipseq_std_sample_name'].apply(get_chipseq_path)
fn = os.path.join(samplesheet_dir, 'chipseq.tracker.hichip_to_chipseq.paths_mapped.{}.tsv'.format(batch))
mapper.to_csv(fn, sep='\t', index=False, header=True)

results/peaks/chipline_v2/BMDC.GSE185880.Mus_Musculus.H3K27ac.b1/MACS2_Ext_*/BMDC.GSE185880.Mus_Musculus.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt
results/peaks/chipline_v2/BMDC.GSE185880.Mus_Musculus.H3K27ac.b1/MACS2_Ext_*/BMDC.GSE185880.Mus_Musculus.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt
results/peaks/chipline_v2/BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1/MACS2_Ext_*/BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt
results/peaks/chipline_v2/BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1/MACS2_Ext_*/BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt
results/peaks/chipline_v2/BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27ac.b1/MACS2_Ext_*/BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt
results/peaks/chipline_v2/BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27ac.b1/MACS2_Ext_*/BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt
results/peaks/chipline_v2/BMDC-LPS.GSE1858